# Classification based on leaves features

### Importing our own functions


In [1]:
import importlib

import src.Data as Data
importlib.reload(Data)
Data = Data.Data

import src.Metrics as Metrics
importlib.reload(Metrics)
Metrics = Metrics.Metrics


### Importing Libraries

`numpy` and `pandas` are used to manipulate the data

`scikit-learn` is used to train the classification models and compute the metrics

In [2]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import LearningCurveDisplay, learning_curve, cross_validate, train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

### Choosing the models

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier


classifiers = [
    DecisionTreeClassifier(), 
    RandomForestClassifier(n_jobs=-1), 
    #BaggingClassifier(n_jobs=-1), 
    LogisticRegression(n_jobs=-1),
    #SVC(),
    #GaussianNB(),
    #SGDClassifier(n_jobs=-1),
    #KNeighborsClassifier(n_jobs=-1),
    #GradientBoostingClassifier(),
    #MLPClassifier(),
    #AdaBoostClassifier()
    
    ]

### Loading the data
The data is loaded from the `data` folder.

Samples are split into a training set and a test set with a custom ratio. Stratified sampling is used to ensure that the proportion of samples in each class is the same in both sets.

In [4]:
data: Data = Data(test_size=0.2, include_images=False)

least_populated_class_count = np.unique(data.y_train, return_counts=True)[1].min()
print("Least populated class count:", least_populated_class_count)


Least populated class count: 8


### Training the models

#### _Without cross-validation_



We first train the models on the training set without cross-validation.

In [5]:
print('Fit the model without cross-validation or hyperparameter tuning. \n')

metrics_list = []

for classifier in classifiers:
    classifier.fit(data.x_train, data.y_train)
    predictions = classifier.predict(data.x_test)

    classifier_metrics = Metrics(classifier_name=classifier.__class__.__name__)
    # Computing metrics
    classifier_metrics.add_metric('accuracy_score', accuracy_score(data.y_test, predictions))
    classifier_metrics.add_metric('f1_score', f1_score(data.y_test, predictions, average='macro'))
    classifier_metrics.add_metric('precision_score', precision_score(data.y_test, predictions, average='macro'))
    classifier_metrics.add_metric('recall_score', recall_score(data.y_test, predictions, average='macro'))

    metrics_list.append(classifier_metrics)

    Metrics.show_metrics_list(metrics_list)

Fit the model without cross-validation or hyperparameter tuning. 

